<a href="https://colab.research.google.com/github/Saurav2910/My-Deeplearning-Projects/blob/main/Heart_Beat_Classification_with_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Heart Beat Classification using CNN

Cardiovascular diseases are one of the leading causes of death worldwide according to WHO. Therefore, being able to detect abnormalities in heartbeats is an important task. 

Out motivation here is to be able to automatically classify and distinguish normal heartbeats from abnormal ones.

The proposed solution is a DL model that can classify real heart audio (also known as “beat classification”) into one of multiple categories: **Normal, Extrahls, Murmur, Artifact or Unlabeled.** 

Heartbeats consist of 2 consecutive noises called ‘lub’ and ‘dub’ which are produced due to the heart movements. 
These noises are of varying frequencies and occur in close proximity in normal situations. 

We aim to use CNN to classify these heartbeats into the aforementioned classes.


In [ ]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [ ]:
!wget https://www.dropbox.com/s/pf2vls3bso2vhjr/set_a.csv

--2020-08-07 16:52:04--  https://www.dropbox.com/s/pf2vls3bso2vhjr/set_a.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/pf2vls3bso2vhjr/set_a.csv [following]
--2020-08-07 16:52:04--  https://www.dropbox.com/s/raw/pf2vls3bso2vhjr/set_a.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1be9044307eb62a367d4b26ad2.dl.dropboxusercontent.com/cd/0/inline/A9D_zS6odTv4dZC1e7xSNiUcyN_3K0tmDXP_OsGkVYZjEiVs8I6vbEUaqP8flV02kDB8mzKUAXCZbsX3iaGyjt_T_NLJAXmGyzXm_Q-D-A7TxRaDInn7xggyJz7D16GKeG0/file# [following]
--2020-08-07 16:52:05--  https://uc1be9044307eb62a367d4b26ad2.dl.dropboxusercontent.com/cd/0/inline/A9D_zS6odTv4dZC1e7xSNiUcyN_3K0tmDXP_OsGkVYZjEiVs8I6vbEUaqP8flV02kDB8mzKUAXCZbsX3iaGyjt_T_NLJAXmGyzXm_Q-D-A7TxRaDInn7xggy

In [ ]:
!wget https://www.dropbox.com/s/d2wgk8j0wsye2ab/set_a.zip

--2020-08-07 16:52:11--  https://www.dropbox.com/s/d2wgk8j0wsye2ab/set_a.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/d2wgk8j0wsye2ab/set_a.zip [following]
--2020-08-07 16:52:11--  https://www.dropbox.com/s/raw/d2wgk8j0wsye2ab/set_a.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucfe21500e63b7dceb3eb55ad0b7.dl.dropboxusercontent.com/cd/0/inline/A9Bb5y9tX4FRzUkAXRjRB4sgQIH1MR1xkG_XFNm9yFvH2IQDnBSUkNEClvnRYppmZv6i-QWqwCz2XskspkwUkkuMW3zY749QbtLWmQFfK3X2V-aAecMmyNojtvUE75xu6t8/file# [following]
--2020-08-07 16:52:12--  https://ucfe21500e63b7dceb3eb55ad0b7.dl.dropboxusercontent.com/cd/0/inline/A9Bb5y9tX4FRzUkAXRjRB4sgQIH1MR1xkG_XFNm9yFvH2IQDnBSUkNEClvnRYppmZv6i-QWqwCz2XskspkwUkkuMW3zY749QbtLWmQFfK3X2V-aAecMmyNoj

In [ ]:
!unzip set_a.zip -d audio

Archive:  set_a.zip
  inflating: audio/artifact__201012172012.wav  
  inflating: audio/artifact__201105040918.wav  
  inflating: audio/artifact__201105041959.wav  
  inflating: audio/artifact__201105051017.wav  
  inflating: audio/artifact__201105060108.wav  
  inflating: audio/artifact__201105061143.wav  
  inflating: audio/artifact__201105190800.wav  
  inflating: audio/artifact__201105280851.wav  
  inflating: audio/artifact__201106010559.wav  
  inflating: audio/artifact__201106010602.wav  
  inflating: audio/artifact__201106021541.wav  
  inflating: audio/artifact__201106030612.wav  
  inflating: audio/artifact__201106031558.wav  
  inflating: audio/artifact__201106040722.wav  
  inflating: audio/artifact__201106040933.wav  
  inflating: audio/artifact__201106040947.wav  
  inflating: audio/artifact__201106041452.wav  
  inflating: audio/artifact__201106050353.wav  
  inflating: audio/artifact__201106061233.wav  
  inflating: audio/artifact__201106070537.wav  
  inflating: audio/a

In [ ]:
import os
import pickle
from glob import iglob
import numpy as np

#LibROSA is a python package for music and audio analysis
import librosa

In [ ]:
DATA_AUDIO_DIR = './audio'
TARGET_SR = 8000
OUTPUT_DIR = './output'
OUTPUT_DIR_TRAIN = os.path.join(OUTPUT_DIR, 'train')
OUTPUT_DIR_TEST = os.path.join(OUTPUT_DIR, 'test')
AUDIO_LENGTH = 10000

In [ ]:
os.makedirs('/content/output/train/')
os.makedirs('/content/output/test/')

## Put Label Index to each Category

In [ ]:
class_ids = {
    'normal': 0,
    'murmur': 1,
    'extrahls': 2,
    'artifact': 3,
    'unlabelled': 4,
}

### Read Category/Labels from the File Names

In [ ]:
def extract_class_id(wav_filename):
    if 'normal' in wav_filename:
        return class_ids.get('normal')
    elif 'murmur' in wav_filename:
        return class_ids.get('murmur')
    elif 'extrahls' in wav_filename:
        return class_ids.get('extrahls')
    elif 'artifact' in wav_filename:
        return class_ids.get('artifact')
    elif 'unlabelled' in wav_filename:
        return class_ids.get('unlabelled')
    else:
        return class_ids.get('unlabelled')

## Load the Audio from the WAV File

In [ ]:
# Load an audio file as a floating point time series.
# Audio will be automatically resampled to the given rate (default sr=22050).
# To preserve the native sampling rate of the file, use sr=None.
def read_audio_from_filename(filename, target_sr):
    audio, _ = librosa.load(filename, sr=target_sr, mono=True)
    audio = audio.reshape(-1, 1)
    return audio

### Custom Function for Audio Formatting

In [ ]:
def convert_data():
    for i, wav_filename in enumerate(iglob(os.path.join(DATA_AUDIO_DIR, '**/**.wav'), recursive=True)):
        class_id = extract_class_id(wav_filename)
        audio_buf = read_audio_from_filename(wav_filename, target_sr=TARGET_SR)
       
        # Normalize mean 0, variance 1
        audio_buf = (audio_buf - np.mean(audio_buf)) / np.std(audio_buf)
        original_length = len(audio_buf)
        print(i, wav_filename, original_length, np.round(np.mean(audio_buf), 4), np.std(audio_buf))
        if original_length < AUDIO_LENGTH:
            audio_buf = np.concatenate((audio_buf, np.zeros(shape=(AUDIO_LENGTH - original_length, 1))))
            print('PAD New length =', len(audio_buf))
        elif original_length > AUDIO_LENGTH:
            audio_buf = audio_buf[0:AUDIO_LENGTH]
            print('CUT New length =', len(audio_buf))

        output_folder = OUTPUT_DIR_TRAIN
        if i // 10 == 0:
            output_folder = OUTPUT_DIR_TEST

        output_filename = os.path.join(output_folder, str(i) + '.pkl')

        out = {'class_id': class_id,
               'audio': audio_buf,
               'sr': TARGET_SR}
        w=open(output_filename,'wb')
        pickle.dump(out,w)
        w.close()
        

In [ ]:
convert_data()

0 ./audio/artifact__201106101955.wav 72000 0.0 1.0
CUT New length = 10000
1 ./audio/normal__201106141148.wav 45995 -0.0 1.0
CUT New length = 10000
2 ./audio/normal__201106221450.wav 72000 -0.0 1.0
CUT New length = 10000
3 ./audio/Aunlabelledtest__201106100606.wav 72000 0.0 1.0
CUT New length = 10000
4 ./audio/Aunlabelledtest__20110501548.wav 72000 0.0 1.0
CUT New length = 10000
5 ./audio/murmur__201102052338.wav 72000 -0.0 1.0
CUT New length = 10000
6 ./audio/normal__201101070538.wav 71073 -0.0 1.0
CUT New length = 10000
7 ./audio/Aunlabelledtest__201106061215.wav 72000 -0.0 1.0
CUT New length = 10000
8 ./audio/artifact__201105280851.wav 72000 -0.0 1.0
CUT New length = 10000
9 ./audio/artifact__201106021541.wav 72000 -0.0 0.99999994
CUT New length = 10000
10 ./audio/normal__201106221418.wav 56023 -0.0 0.99999994
CUT New length = 10000
11 ./audio/Aunlabelledtest__201106040930.wav 72000 -0.0 1.0
CUT New length = 10000
12 ./audio/artifact__201106220340.wav 72000 -0.0 1.0
CUT New length = 

### Build the ConvNet Model

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Lambda,Conv1D, MaxPooling1D,Activation, Dense,BatchNormalization

from tensorflow.keras.models import Sequential
import numpy as np
import pickle
import os
from glob import glob

In [ ]:
AUDIO_LENGTH = 10000
OUTPUT_DIR = './output'
OUTPUT_DIR_TRAIN = os.path.join(OUTPUT_DIR, 'train')
OUTPUT_DIR_TEST = os.path.join(OUTPUT_DIR, 'test')

In [ ]:
def m5(num_classes=5):
    print('Using Model M5')
    m = Sequential()
    m.add(Conv1D(128,
                 input_shape=[AUDIO_LENGTH, 1],
                 kernel_size=80,
                 strides=4,
                 padding='same',
                 kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001)))
    m.add(BatchNormalization())
    m.add(Activation('relu'))
    m.add(MaxPooling1D(pool_size=4, strides=None))
    m.add(Conv1D(128,
                 kernel_size=3,
                 strides=1,
                 padding='same',
                 kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001)))
    m.add(BatchNormalization())
    m.add(Activation('relu'))
    m.add(MaxPooling1D(pool_size=4, strides=None))
    m.add(Conv1D(256,
                 kernel_size=3,
                 strides=1,
                 padding='same',
                 kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001)))
    m.add(BatchNormalization())
    m.add(Activation('relu'))
    m.add(MaxPooling1D(pool_size=4, strides=None))
    m.add(Conv1D(512,
                 kernel_size=3,
                 strides=1,
                 padding='same',
                 kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001)))
    m.add(BatchNormalization())
    m.add(Activation('relu'))
    m.add(MaxPooling1D(pool_size=4, strides=None))
    m.add(Lambda(lambda x: K.mean(x, axis=1)))  
    m.add(Dense(num_classes, activation='softmax'))
    return m

### Custom Function - Get Data in Training Data and Label Format

In [ ]:
def get_data(file_list):
    def load_into(_filename, _x, _y):
        with open(_filename, 'rb') as f:
            audio_element = pickle.load(f)
            _x.append(audio_element['audio'])
            _y.append(int(audio_element['class_id']))

    x, y = [], []
    for filename in file_list:
        load_into(filename, x, y)
    return np.array(x), np.array(y)

In [ ]:
num_classes = 5
model = m5(num_classes=num_classes)

if model is None:
    exit('Something went wrong!!')

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
print(model.summary())

# Read Training data
train_files = glob(os.path.join(OUTPUT_DIR_TRAIN, '**.pkl'))
x_tr, y_tr = get_data(train_files)
y_tr = to_categorical(y_tr, num_classes=num_classes)

# Read Test Data
test_files = glob(os.path.join(OUTPUT_DIR_TEST, '**.pkl'))
x_te, y_te = get_data(test_files)
y_te = to_categorical(y_te, num_classes=num_classes)

print('x_tr.shape =', x_tr.shape)
print('y_tr.shape =', y_tr.shape)
print('x_te.shape =', x_te.shape)
print('y_te.shape =', y_te.shape)

# Define a callback
# if the accuracy does not increase over 10 epochs, reduce the learning rate by half.
reduce_lr = ReduceLROnPlateau(monitor='acc', factor=0.5, patience=10, min_lr=0.0001, verbose=1)

# Set a batch size
batch_size = 128

# Train the model
model.fit(x=x_tr,
          y=y_tr,
          batch_size=batch_size,
          epochs=400,
          verbose=1,
          shuffle=True,
          validation_data=(x_te, y_te),
          callbacks=[reduce_lr])

Using Model M5
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 2500, 128)         10368     
_________________________________________________________________
batch_normalization (BatchNo (None, 2500, 128)         512       
_________________________________________________________________
activation (Activation)      (None, 2500, 128)         0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 625, 128)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 625, 128)          49280     
_________________________________________________________________
batch_normalization_1 (Batch (None, 625, 128)          512       
_________________________________________________________________
activation_1 (Activation)    (None, 625, 